# Special kursus

## Import of modules

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=f'{0}'
from __future__ import annotations

import torch

from modules import dataHandler, dataprocessing, models
from modules import scalers as scaling

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"cuda {torch.cuda.current_device()}")
else:
    print("cpu")
    device = torch.device("cpu")

cuda 0


## Load data

Loads all the different datasets: Regions, TS profiles, Bathymetri, GHRSST and SMOS.\
The TS profiles also gets cleaned and split into different regions

In [ ]:
(
    regions,
    ts_profiles,
    bathymetri_lat, bathymetri_lon, bathymetri_topography,
    ghrsst_lat, ghrsst_lon, ghrsst_sst, ghrsst_times, ghrsst_fraction_time, ghrsst_time_bnds,
    smos_sss, smos_time, smos_lat, smos_lon
) = dataHandler.load_all(verbose=True)

In [ ]:
ts_areas = dataprocessing.split_regions(ts_profiles, regions.set_crs(4326))

## Creation of training, validation and testing datasets

The following code splits the dataset into 3 datasets (training, valdiation and testing).\
The data outside the ghrsst time is cut.

In [5]:
min_time, max_time = min(ghrsst_fraction_time), max(ghrsst_fraction_time)
training_end = 2012
validation_end = 2015
train, val, test, areas = dataprocessing.split_data_set(ts_areas, training_end, validation_end, min_time, max_time)

# Training

Converts the dict objects into dataloaders containing torch tensors and scales each feature

In [6]:
scalers = [
    scaling.MeanScaling, # lat
    scaling.MeanScaling, # lon
    scaling.MeanScaling, # year
    scaling.MeanScaling, # decimal year
    scaling.MeanScaling, # sss
    scaling.MeanScaling, # sst
    scaling.MeanScaling, # surface depth
    scaling.MinMaxScaling, # bathymetri
    scaling.MeanScaling, # salinity profile
    scaling.MeanScaling, # temperature profile
]

scalers, train_loader, val_loader, test_loader = dataprocessing.process_data(
    train=train,
    val=val,
    test=test,
    bathymetri_lat=bathymetri_lat,
    bathymetri_lon=bathymetri_lon,
    bathymetri_topography=bathymetri_topography,
    scalers= scalers
)

Trains the model and saves it

In [7]:
training_loss, validation_loss = models.train_model(train_loader, val_loader)

Model:
	RNN:
		Input 8, Output 21, Layers 2
	Linear:
		Input 21, Output 21, Layers 1
Optimizer:
	lr 0.5, momentum 0.8 weight decay 0.0001

[   1/4000] Salinity Training Loss 1.65832, lr 0.500 | Validation Loss 1.34949, MedAE 1.164
[  10/4000] Salinity Training Loss 0.75444, lr 0.490 | Validation Loss 0.47576, MedAE 0.327
[  20/4000] Salinity Training Loss 0.61670, lr 0.457 | Validation Loss 0.45778, MedAE 0.304
[  30/4000] Salinity Training Loss 0.57101, lr 0.403 | Validation Loss 0.45115, MedAE 0.292
[  40/4000] Salinity Training Loss 0.57721, lr 0.335 | Validation Loss 0.45064, MedAE 0.297
[  50/4000] Salinity Training Loss 0.52660, lr 0.258 | Validation Loss 0.45788, MedAE 0.300
[  60/4000] Salinity Training Loss 0.49775, lr 0.181 | Validation Loss 0.45000, MedAE 0.293
[  70/4000] Salinity Training Loss 0.48322, lr 0.110 | Validation Loss 0.44146, MedAE 0.280
[  80/4000] Salinity Training Loss 0.47580, lr 0.053 | Validation Loss 0.43780, MedAE 0.274
[  90/4000] Salinity Training Los

[ 880/4000] Salinity Training Loss 0.52831, lr 0.053 | Validation Loss 0.45509, MedAE 0.312
[ 890/4000] Salinity Training Loss 0.52804, lr 0.016 | Validation Loss 0.45490, MedAE 0.312
[ 900/4000] Salinity Training Loss 0.52797, lr 0.001 | Validation Loss 0.45489, MedAE 0.312
[ 910/4000] Salinity Training Loss 0.53286, lr 0.490 | Validation Loss 0.45351, MedAE 0.310
[ 920/4000] Salinity Training Loss 0.52609, lr 0.457 | Validation Loss 0.45400, MedAE 0.308
[ 930/4000] Salinity Training Loss 0.52913, lr 0.403 | Validation Loss 0.51076, MedAE 0.316
[ 940/4000] Salinity Training Loss 0.51858, lr 0.335 | Validation Loss 0.44197, MedAE 0.304
[ 950/4000] Salinity Training Loss 0.47965, lr 0.258 | Validation Loss 0.47563, MedAE 0.312
[ 960/4000] Salinity Training Loss 0.47008, lr 0.181 | Validation Loss 0.46786, MedAE 0.301
[ 970/4000] Salinity Training Loss 0.46792, lr 0.110 | Validation Loss 0.47476, MedAE 0.301
[ 980/4000] Salinity Training Loss 0.46700, lr 0.053 | Validation Loss 0.47644, 

[1780/4000] Salinity Training Loss 0.49797, lr 0.053 | Validation Loss 0.44137, MedAE 0.306
[1790/4000] Salinity Training Loss 0.49776, lr 0.016 | Validation Loss 0.44129, MedAE 0.306
[1800/4000] Salinity Training Loss 0.49771, lr 0.001 | Validation Loss 0.44126, MedAE 0.306
[1810/4000] Salinity Training Loss 0.54980, lr 0.490 | Validation Loss 0.45474, MedAE 0.326
[1820/4000] Salinity Training Loss 0.51286, lr 0.457 | Validation Loss 0.44135, MedAE 0.305
[1830/4000] Salinity Training Loss 0.49578, lr 0.403 | Validation Loss 0.43711, MedAE 0.307
[1840/4000] Salinity Training Loss 0.49210, lr 0.335 | Validation Loss 0.43601, MedAE 0.302
[1850/4000] Salinity Training Loss 0.49057, lr 0.258 | Validation Loss 0.43634, MedAE 0.303
[1860/4000] Salinity Training Loss 0.48951, lr 0.181 | Validation Loss 0.43551, MedAE 0.302
[1870/4000] Salinity Training Loss 0.48882, lr 0.110 | Validation Loss 0.43528, MedAE 0.302
[1880/4000] Salinity Training Loss 0.48843, lr 0.053 | Validation Loss 0.43513, 

[2680/4000] Salinity Training Loss 0.46288, lr 0.053 | Validation Loss 0.41380, MedAE 0.280
[2690/4000] Salinity Training Loss 0.46270, lr 0.016 | Validation Loss 0.41376, MedAE 0.280
[2700/4000] Salinity Training Loss 0.46266, lr 0.001 | Validation Loss 0.41373, MedAE 0.280
[2710/4000] Salinity Training Loss 0.54607, lr 0.490 | Validation Loss 0.47946, MedAE 0.333
[2720/4000] Salinity Training Loss 0.51144, lr 0.457 | Validation Loss 0.44637, MedAE 0.309
[2730/4000] Salinity Training Loss 0.49374, lr 0.403 | Validation Loss 0.44309, MedAE 0.301
[2740/4000] Salinity Training Loss 0.48720, lr 0.335 | Validation Loss 0.43567, MedAE 0.297
[2750/4000] Salinity Training Loss 0.48395, lr 0.258 | Validation Loss 0.43525, MedAE 0.297
[2760/4000] Salinity Training Loss 0.48201, lr 0.181 | Validation Loss 0.43477, MedAE 0.296
[2770/4000] Salinity Training Loss 0.48083, lr 0.110 | Validation Loss 0.43466, MedAE 0.296
[2780/4000] Salinity Training Loss 0.48019, lr 0.053 | Validation Loss 0.43459, 

[3580/4000] Salinity Training Loss 0.50235, lr 0.053 | Validation Loss 0.43678, MedAE 0.299
[3590/4000] Salinity Training Loss 0.50205, lr 0.016 | Validation Loss 0.43662, MedAE 0.299
[3600/4000] Salinity Training Loss 0.50198, lr 0.001 | Validation Loss 0.43660, MedAE 0.299
[3610/4000] Salinity Training Loss 0.77918, lr 0.490 | Validation Loss 1.05507, MedAE 0.669
[3620/4000] Salinity Training Loss 0.63478, lr 0.457 | Validation Loss 0.48305, MedAE 0.325
[3630/4000] Salinity Training Loss 0.58339, lr 0.403 | Validation Loss 0.44971, MedAE 0.311
[3640/4000] Salinity Training Loss 0.56214, lr 0.335 | Validation Loss 0.46706, MedAE 0.327
[3650/4000] Salinity Training Loss 0.55032, lr 0.258 | Validation Loss 0.44837, MedAE 0.315
[3660/4000] Salinity Training Loss 0.54662, lr 0.181 | Validation Loss 0.45213, MedAE 0.317
[3670/4000] Salinity Training Loss 0.54534, lr 0.110 | Validation Loss 0.45347, MedAE 0.318
[3680/4000] Salinity Training Loss 0.54449, lr 0.053 | Validation Loss 0.45388, 